# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [99]:
import numpy as np
import pandas as pd
import xlsxwriter
import math
import requests
import yfinance as yf

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [100]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,A,Agilent Technologies,Health Care,Life Sciences Tools & Services,"Santa Clara, California",05/06/00,1090872,1999
1,AAPL,Apple Inc.,Information Technology,"Technology Hardware, Storage & Peripherals","Cupertino, California",30/11/82,320193,1977
2,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",31/12/12,1551152,2013 (1888)
3,ABNB,Airbnb,Consumer Discretionary,"Hotels, Resorts & Cruise Lines","San Francisco, California",18/09/23,1559720,2008
4,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",04/03/57,1800,1888
...,...,...,...,...,...,...,...,...
498,XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components,"White Plains, New York",01/11/11,1524472,2011
499,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",06/10/97,1041061,1997
500,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",07/08/01,1136869,1927
501,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",23/12/19,877212,1969


## Making Our First API Call

We need the following information from the yfinance API:

* Price of each stock



In [101]:
symbol = 'AAPL'
stock = yf.Ticker(symbol)

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [102]:
stock_price = stock.history(period="1d")['Close'].iloc[-1]
print(stock_price)

224.22999572753906


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [103]:
cols = [ 'Ticker', 'Stock Price', 'Number of Shares to Buy' ]
final_df = pd.DataFrame(columns = cols)
final_df

,Ticker,Stock Price,Number of Shares to Buy


In [104]:
new_row = pd.DataFrame([[symbol, stock_price, 'N/A']], columns=cols)


final_df = pd.concat([final_df, new_row])
final_df

/var/folders/hd/h4ty4f0j2tgfss4vb7rrr93w0000gn/T/ipykernel_24002/3764206733.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat([final_df, new_row])


,Ticker,Stock Price,Number of Shares to Buy
0,AAPL,224.229996,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [105]:
final_df = pd.DataFrame(columns=cols)
for st in stocks['Symbol']:
    stock = yf.Ticker(st)
    stock_price = stock.history(period="1d")['Close'].iloc[-1]

    new_row = pd.DataFrame([[st, stock_price, 'N/A']], columns=cols)

    final_df = pd.concat([final_df, new_row], ignore_index=True)
final_df

/var/folders/hd/h4ty4f0j2tgfss4vb7rrr93w0000gn/T/ipykernel_24002/3621518741.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat([final_df, new_row], ignore_index=True)


,Ticker,Stock Price,Number of Shares to Buy
0,A,133.669998,N/A
1,AAPL,224.229996,N/A
2,ABBV,171.089996,N/A
3,ABNB,133.199997,N/A
4,ABT,116.440002,N/A
...,...,...,...
498,XYL,124.790001,N/A
499,YUM,134.610001,N/A
500,ZBH,109.940002,N/A
501,ZBRA,402.690002,N/A


## Using Batch API Calls to Improve Performance

In [106]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [107]:
symbol_grps = list(chunks(stocks['Symbol'], 100))
symbol_strings = []

for i in range(0, len(symbol_grps)):
    symbol_strings.append(' '.join(symbol_grps[i]))

final_df = pd.DataFrame(columns=cols)

for symbol_string in symbol_strings:
    batch_api_call = yf.download(symbol_string, period="1d")

    for symbol in symbol_string.split(' '):
        price = batch_api_call['Close'][symbol].iloc[-1]
        new_row = pd.DataFrame([[symbol, batch_api_call['Close'][symbol].iloc[-1], 'N/A']], columns=cols)

        final_df = pd.concat([final_df, new_row], ignore_index=True)

final_df

[*********************100%***********************]  100 of 100 completed
/var/folders/hd/h4ty4f0j2tgfss4vb7rrr93w0000gn/T/ipykernel_24002/57352750.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat([final_df, new_row], ignore_index=True)
[*********************100%***********************]  100 of 100 completed
Exception ignored in: <function ZipFile.__del__ at 0x103b239c0>
Traceback (most recent call last):
  File "/opt/miniconda3/envs/algo_trading/lib/python3.11/zipfile.py", line 1895, in __del__
    self.close()
  File "/opt/miniconda3/envs/algo_trading/lib/python3.11/zipfile.py", line 1912, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file
Exception ignored in: <function ZipFile.__del__ at 0x

,Ticker,Stock Price,Number of Shares to Buy
0,A,133.669998,N/A
1,AAPL,224.229996,N/A
2,ABBV,171.089996,N/A
3,ABNB,133.199997,N/A
4,ABT,116.440002,N/A
...,...,...,...
498,XYL,124.790001,N/A
499,YUM,134.610001,N/A
500,ZBH,109.940002,N/A
501,ZBRA,402.690002,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [108]:
portfolio_size = input('Enter the value of your portfolio: ')

try:
    val = float(portfolio_size)
except ValueError:
    print("Please enter an integer")
    portfolio_size = input('Enter the value of your portfolio: ')
    val = float(portfolio_size)

In [109]:
position_size = val/len(final_df.index)

for i in range(0, len(final_df.index)):
    price = final_df.loc[i, 'Stock Price']
    final_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / price)

final_df

,Ticker,Stock Price,Number of Shares to Buy
0,A,133.669998,14
1,AAPL,224.229996,8
2,ABBV,171.089996,11
3,ABNB,133.199997,14
4,ABT,116.440002,17
...,...,...,...
498,XYL,124.790001,15
499,YUM,134.610001,14
500,ZBH,109.940002,18
501,ZBRA,402.690002,4


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

### Initializing our XlsxWriter Object

In [110]:
writer = pd.ExcelWriter('recommended trades.xlsx', engine='xlsxwriter')
final_df.to_excel(writer, 'Recommended Trades', index=False)

/var/folders/hd/h4ty4f0j2tgfss4vb7rrr93w0000gn/T/ipykernel_24002/3571317376.py:2: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  final_df.to_excel(writer, 'Recommended Trades', index=False)


### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* Integer format for the number of shares to purchase

In [111]:
background_color = '#0a0a23'
font_color = 'ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

In [112]:
col_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Number of Shares to Buy', integer_format]
}

for col in col_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{col}:{col}', 18, col_formats[col][1])
    writer.sheets['Recommended Trades'].write(f'{col}1:{col}1', col_formats[col][0], col_formats[col][1])


## Saving Our Excel Output

In [113]:
writer.close()